In [2]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from glob import glob
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torchsummary import summary
import torch.nn.functional as F
import torch.utils.data as utils
from tqdm import tqdm
from os.path import isfile, join
from os import listdir
import datetime
import pandas as pd
import time, copy
from models_pytorch import CNNT5, CNNT4
from vgg_pytorch import vgg11
from data_preprocessing.dataloaders_pytorch import LUNA_Dataset
from test_tools import model_predict

LOGS = '/home.stud/dobkomar/NoduleDetection/logs/'


def get_free_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print(memory_available)
    os.remove('tmp')
    return np.argmax(memory_available)

In [3]:
data = pd.read_csv('/home.stud/dobkomar/data/test_D1.csv')


# IF 2-D DATASET
test_dataset = LUNA_Dataset(data)

# ELIF 3-D DATASET
# test_dataset = LUNA_Dataset_3D(data)

test_loader = utils.DataLoader(test_dataset, batch_size=1, shuffle=False)

### set GPU mode

In [4]:
device = torch.device("cuda:{}".format(get_free_gpu()))

[10659, 259, 11168, 11168, 395, 395, 395, 395]


# Prediction

In [5]:
# IF VGG:

# model_conv = torchvision.models.vgg11(pretrained=False)

# num_features = model_conv.classifier[6].in_features
# features = list(model_conv.classifier.children())[:-1]
# model_conv = model_conv.to(device)
# features[0] = nn.Linear(512, 4096)
# features.extend([nn.Linear(4096, 2)])
# model_conv.classifier = nn.Sequential(*features)


# ALL OTHER MODELS

model_conv = CNNT4()

path_weights = LOGS + 'CNNT4_D1.pt'
model_conv.load_state_dict(torch.load(path_weights))
model_conv.to(device)


TEST = 'test'
image_datasets = {
    'test': test_dataset
}
dataloaders_dict = {'test': test_loader}
dataset_sizes = {x: len(image_datasets[x]) for x in [TEST]}

recall_test, acc_test = predict_model(model_conv, dataloaders_dict)
print([recall_test, acc_test])
